In [254]:
import random
import numpy as np

class GeneticAlgorithm:
    #Here we get our parameters and our input
    def __init__(self, stock_length, requests, target, population_size, mutation_rate):
        self.stock_length = stock_length
        self.requests = requests
        self.target = target
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.build_population()
    #Here we build our initial population randomly
    def build_population(self):
        self.population = [self.requests.copy()]
        for i in range(self.population_size - 1):
            self.population.append(random.sample(self.requests, len(self.requests)))
    #Here we calculate how much of a waste we will have if we cut in our gene's order
    def fitness(self, gen):
        waste = 0
        left = self.stock_length
        for i in range(len(gen)):
            cur = gen[i]
            if(cur <= left):
                left -= cur
            else:
                waste += left
                left = self.stock_length - cur
        waste += left
        
        return waste
    #Here we update our weights because our population is changed
    def update_weights(self):
        fitness_scores = [self.fitness(gen) for gen in self.population]
        max_fitness = max(fitness_scores)
        fitness_scores = [max_fitness - fitness_score + 1 for fitness_score in fitness_scores]
        self.weights = fitness_scores
        return
    #Here we find parents to crossover based on max_fitness - their fitness probability
    def select_parents(self):
        parents = random.choices(self.population, weights=self.weights, k=2)
        return parents
    #Here we find the best two parents to keep them from a generation to next
    def select_best_parents(self):
        mx1 = -1; mx1id = -1
        mx2 = -1; mx2id = -1
        for i in range(len(self.weights)):
            if(self.weights[i] > mx1):
                mx2 = mx1; mx2id = mx1id
                mx1 = self.weights[i]; mx1id = i
            elif(self.weights[i] > mx2):
                mx2 = self.weights[i]; mx2id = i
            
        parents = [self.population[mx1id], self.population[mx2id]]
        return parents
    #Here we get two parents and we find their child (first parents first half and second parents leftovers)    
    #Based on two pointer algorithm
    def crossover(self, parents):
        child = []
        gen1 = [(parents[0][i], i) for i in range(len(parents[0]))]
        gen1.sort()
        gen2 = [(parents[1][i], i) for i in range(len(parents[1]))]
        gen2.sort()
        
        pnt1 = 0
        pnt2 = 0
        while(pnt1 < len(gen1)):
            if(gen1[pnt1][1] >= (len(gen1) // 2)):
                pnt1 += 1
                continue
                
            while(gen2[pnt2][0] < gen1[pnt1][0]):
                gen2[pnt2] = (gen2[pnt2][1], gen2[pnt2][0])
                pnt2 += 1
            
            gen2[pnt2] = (-1, -1)
            pnt2 += 1
            pnt1 += 1
        while(pnt2 < len(gen2)):
            gen2[pnt2] = (gen2[pnt2][1], gen2[pnt2][0])
            pnt2 += 1
        
        gen2.sort()
        for i in range(len(gen2)):
            if(gen2[i][0] != -1):
                 child.append(gen2[i][1])
        
        for i in range(len(parents[0]) // 2):
            child.append(parents[0][i])
            
        return child
    #Here we mutate a gene to a new random gene
    def mutate(self, gen):
        if random.random() <= self.mutation_rate:
            gen = random.sample(self.requests, len(self.requests))

        return gen
    #Here we make new generations and we search if we find a solution
    def evolve(self, num_generations):
        best_gen = []
        best_rolls_num = 100000000000000
        for generation in range(num_generations):
            if(generation % 100 == 0):
                print(generation, best_rolls_num)
                
            self.update_weights()
            
            for gen in self.population:
                rolls_num = (sum(gen) + self.fitness(gen)) / self.stock_length
                if(rolls_num < best_rolls_num):
                    best_rolls_num = rolls_num
                    best_gen = gen
                
                if self.fitness(gen) <= self.target:
                    print("Found Answer in generation:", generation, "With gen:", gen, "and use of", rolls_num, "rolls.")
                    return
            
            children = self.select_best_parents()
            for i in range(self.population_size - 2):
                new_child = self.mutate(
                                self.crossover(
                                    self.select_parents()))
                if(new_child in children):
                    new_child = random.sample(self.requests, len(self.requests))
                    
                children.append(new_child)
            
            self.population = children
        print("best solution with gen of = ", best_gen, " and number of rolls of:", best_rolls_num)

In [257]:
import re
#Here we read all our inputs and based on our prameter sets
#we run our genetic algorithm on them and find best results
targets = [55, 79, 114, 234]
for i in range(4):
    with open('input' + chr(49 + i) + '.stock', 'r') as file:
        inp = file.read()
        inp = re.split(r'[,\s\n]+', inp)
        
        stock_length = int(inp[2])
        requests = [int(inp[j]) for j in range(inp.index('Requests:') + 1, inp.index('Answer:'))]
        requests.sort()
        target = targets[i] * stock_length - sum(requests)
        
        A = [(100, 0.02, 3000), (100, 0.03, 3000), (200, 0.02, 1000), (200, 0.03, 1000)]
        
        for (population_size, mutation_rate, num_generations) in A:
            print("population_size = ", population_size)
            print("mutation_rate = ", mutation_rate)
            print("num_generations = ", num_generations)
            ga = GeneticAlgorithm(stock_length, requests, target, population_size, mutation_rate)
            ga.evolve(num_generations)

population_size =  100
mutation_rate =  0.02
num_generations =  3000
0 100000000000000
Found Answer in generation: 98 With gen: [241, 618, 109, 501, 117, 116, 33, 506, 463, 84, 662, 79, 266, 301, 251, 592, 283, 264, 286, 266, 555, 351, 69, 414, 286, 517, 211, 99, 88, 805, 678, 119, 148, 653, 337, 315, 144, 457, 230, 609, 115, 354, 60, 187, 284, 268, 78, 248, 135, 18, 868, 88, 933, 914, 987, 672, 988, 544, 346, 249, 370, 53, 71, 92, 106, 753, 125, 106, 149, 557, 660, 295, 412, 405, 518, 280, 716, 145, 232, 686, 437, 532, 292, 107, 581, 967, 46, 186, 632, 218, 507, 171, 368, 80, 495, 409, 460, 43, 788, 23, 515, 70, 333, 689, 312, 246, 126, 371, 365, 180, 402, 627, 181, 424, 549, 441, 525, 788, 61, 75, 648, 86, 278, 45, 118, 224, 170, 123, 306, 356, 312, 149, 557, 92, 106, 753, 125, 106, 149, 557] and use of 55.0 rolls.
population_size =  100
mutation_rate =  0.03
num_generations =  3000
0 100000000000000
100 56.0
200 56.0
Found Answer in generation: 241 With gen: [988, 23, 86, 135, 78, 6

100 83.0
200 83.0
300 82.0
400 82.0
500 82.0
600 82.0
700 82.0
800 82.0
900 82.0
best solution with gen of =  [2200, 2200, 2200, 2200, 2200, 2200, 2200, 2200, 2200, 2200, 2200, 1880, 1380, 2200, 1380, 1380, 1380, 2050, 2050, 1880, 1710, 1880, 1520, 1930, 1520, 1520, 2150, 1820, 1820, 2140, 1560, 1560, 2050, 1710, 1820, 1880, 1710, 2100, 2100, 2150, 1520, 1930, 2050, 1710, 1710, 1380, 2050, 2150, 2150, 2150, 2200, 1930, 2140, 1520, 1880, 1880, 2100, 1380, 2200, 2140, 1710, 1380, 2140, 1380, 1930, 2150, 2100, 1380, 1820, 2050, 2100, 1560, 1930, 2050, 1880, 1930, 1930, 1820, 1710, 1560, 1930, 1820, 1560, 2140, 1880, 1820, 2140, 1520, 1520, 2000, 1520, 1520, 2150, 2150, 1880, 1520, 1930, 1520, 1380, 1930, 1880, 1520, 2140, 1380, 1820, 2100, 2100, 1380, 2200, 2150, 2000, 1520, 1520, 1520, 2150, 1930, 2150, 2000, 2100, 1520, 1930, 2000, 1930, 2000, 2000, 2000, 2000, 1560, 1560, 1820, 2140, 2000, 2140, 1820, 1880, 1820, 1560, 1710, 1560, 2050, 1710, 1380, 1820, 1820, 1880, 1710, 2100, 1520, 2

0 100000000000000
Found Answer in generation: 0 With gen: [6, 54, 1, 9, 14, 2, 1, 15, 5, 225, 52, 239, 2, 93, 17, 1, 2, 10, 18, 6, 172, 5, 2, 12, 7, 2, 11, 1, 133, 15, 3, 9, 147, 9, 360, 68, 228, 243, 7, 4, 1, 3, 8, 214, 16, 14, 1, 2, 7, 288, 32, 43, 1, 8, 4, 144, 6, 8, 364, 6, 34, 14, 29, 13, 3, 281, 3, 10, 314, 17, 4, 1, 470, 7, 225, 9, 1, 2, 13, 8, 204, 14, 23, 3, 27, 1, 13, 1, 138, 1, 3, 41, 419, 3, 43, 3, 5, 3, 4, 3, 5, 12, 4, 2, 339, 6, 2, 7, 152, 3, 1, 2, 2, 9, 1, 299, 11, 131, 108, 1, 209, 22, 7, 7, 275, 1, 2, 61, 152, 6, 7, 1, 21, 4, 5, 158, 5, 14, 118, 6, 1, 9, 166, 2, 9, 264, 7, 8, 5, 4, 3, 25, 87, 311, 4, 1, 255, 3, 5, 5, 4, 4, 10, 3, 134, 90, 255, 2, 264, 10, 3, 198, 3, 3, 23, 124, 8, 12, 1, 2, 251, 3, 6, 38, 1, 10, 2, 11, 4, 38, 174, 6, 4, 16, 27, 14, 41, 138, 12, 13, 1, 9, 2, 2, 201, 10, 189, 18, 6, 2, 45, 399, 3, 7, 111, 102, 2, 2, 10, 1, 2, 8, 28, 1, 1, 199, 9, 11, 170, 245, 161, 6, 156, 3, 1, 2, 3, 1, 170, 2, 5, 1, 169, 7, 120, 6, 3, 1, 9, 1, 7, 12, 6, 9, 14, 4, 8, 2,

100 238.0
200 237.0
300 235.0
Found Answer in generation: 389 With gen: [93, 95, 70, 76, 67, 31, 48, 67, 31, 50, 48, 75, 16, 67, 31, 65, 16, 42, 53, 31, 8, 4, 4, 4, 9, 9, 4, 39, 16, 22, 16, 54, 22, 18, 4, 27, 18, 9, 35, 93, 15, 27, 4, 16, 38, 47, 74, 65, 9, 1, 47, 22, 52, 16, 26, 4, 31, 1, 10, 1, 4, 4, 27, 22, 14, 22, 15, 10, 25, 15, 5, 8, 14, 5, 27, 4, 10, 26, 18, 27, 16, 5, 6, 84, 4, 6, 1, 54, 18, 16, 15, 81, 80, 98, 52, 33, 54, 33, 52, 23, 33, 47, 74, 41, 50, 51, 45, 55, 24, 58, 36, 51, 21, 24, 11, 25, 5, 92, 60, 19, 30, 19, 11, 11, 27, 23, 66, 19, 32, 38, 73, 5, 5, 32, 14, 5, 44, 28, 44, 44, 50, 8, 3, 3, 68, 8, 3, 7, 9, 8, 8, 62, 89, 91, 35, 40, 18, 23, 18, 56, 35, 18, 6, 13, 28, 6, 12, 12, 12, 61, 37, 37, 13, 37, 68, 1, 1, 1, 10, 36, 7, 7, 17, 2, 2, 15, 2, 2, 39, 61, 6, 74, 58, 9, 6, 7, 6, 6, 47, 7, 22, 6, 24, 24, 39, 4, 13, 4, 14, 6, 6, 5, 1, 10, 31, 10, 9, 2, 2, 13, 26, 4, 33, 5, 16, 5, 1, 1, 10, 10, 4, 14, 5, 9, 16, 5, 6, 15, 4, 5, 26, 14, 13, 15, 22, 15, 6, 43, 3, 14, 33, 34, 

In [367]:
class HillClimbing:
    #Here we get our parameters and our input
    def __init__(self, stock_length, initial_solution, search_num, max_iterations):
        self.stock_length = stock_length
        self.current_solution = initial_solution
        self.search_num = search_num
        self.max_iterations = max_iterations
    #Here we calculate how much of a waste we will have if we cut in our point's order
    def fitness(self, current_solution):
        waste = 0
        left = self.stock_length
        for i in range(len(current_solution)):
            cur = current_solution[i]
            if(cur <= left):
                left -= cur
            else:
                waste += left
                left = self.stock_length - cur
        waste += left
        
        return waste
    #Here we search all neighbors of a point and we return first better point we get
    def best_neighbor_main(self, current_solution):
        current_solution_fitness = self.fitness(current_solution)
        for i in range(len(current_solution)):
            for j in range(i + 1, len(current_solution)):
                current_solution[i], current_solution[j] = current_solution[j], current_solution[i]
                if(self.fitness(current_solution) < current_solution_fitness):
                    return current_solution
                current_solution[i], current_solution[j] = current_solution[j], current_solution[i]
         
        return current_solution
    #Here we do as above, but instead we seach only search_num times
    def best_neighbor(self, current_solution):
        current_solution_fitness = self.fitness(current_solution)
        for _ in range(self.search_num):
            i = random.randint(0, len(current_solution) - 1)
            j = random.randint(0, len(current_solution) - 1)
            current_solution[i], current_solution[j] = current_solution[j], current_solution[i]
            if(self.fitness(current_solution) < current_solution_fitness):
                return current_solution
            current_solution[i], current_solution[j] = current_solution[j], current_solution[i]
         
        return current_solution
    #Here we keep going to better points untill we reach our iteration maximum number
    def run(self):
        for iteration in range(self.max_iterations):
            current_value = self.fitness(self.current_solution)
            next_solution = self.best_neighbor(self.current_solution)
            next_value = self.fitness(next_solution)

            if next_value <= current_value:
                self.current_solution = next_solution
            else:
                break
        return self.current_solution, self.fitness(self.current_solution)


In [369]:
import re
#Here we read all our inputs and based on our prameter sets
#we run our hill climbing algorithm on them and find best results
targets = [55, 79, 114, 234]
for i in range(4):
    with open('input' + chr(49 + i) + '.stock', 'r') as file:
        inp = file.read()
        inp = re.split(r'[,\s\n]+', inp)
        
        stock_length = int(inp[2])
        requests = [int(inp[j]) for j in range(inp.index('Requests:') + 1, inp.index('Answer:'))]
        
        A = [(20, 300, 200), (40, 200, 300), (60, 150, 400)]
        
        for (shuffle_num, search_num, max_iterations) in A:
            best_value = 10000000
            best_solution = []
            for i in range(shuffle_num):
                hill_climbing = HillClimbing(stock_length, requests, search_num, max_iterations)
                final_solution, final_value = hill_climbing.run()
                final_value = (final_value + sum(requests)) / stock_length
                if(final_value < best_value):
                    best_value = final_value
                    best_solution = final_solution
                random.shuffle(requests)
            
            print("shuffle_num = ", shuffle_num, "search_num = ", search_num, "max_iterations = ", max_iterations)
            print("best_value = ", best_value, best_solution)
            

shuffle_num =  20 search_num =  300 max_iterations =  200
best_value =  53.0 [557, 365, 148, 278, 506, 914, 88, 460, 402, 218, 515, 301, 118, 84, 86, 171, 412, 211, 627, 618, 805, 988, 716, 248, 241, 116, 99, 126, 495, 457, 69, 181, 351, 405, 71, 525, 557, 144, 230, 501, 170, 662, 280, 45, 312, 295, 125, 507, 424, 409, 517, 23, 686, 356, 354, 115, 246, 186, 61, 557, 292, 135, 149, 678, 346, 788, 632, 75, 788, 88, 43, 967, 532, 268, 284, 312, 370, 106, 868, 753, 125, 180, 441, 92, 672, 933, 33, 266, 286, 337, 414, 149, 123, 264, 660, 371, 53, 80, 544, 283, 149, 106, 753, 463, 286, 581, 437, 592, 653, 18, 106, 107, 549, 648, 518, 689, 92, 306, 145, 251, 109, 315, 249, 117, 224, 60, 187, 78, 555, 368, 79, 333, 232, 106, 266, 70, 609, 46, 987, 119]
shuffle_num =  40 search_num =  200 max_iterations =  300
best_value =  53.0 [80, 337, 286, 301, 264, 689, 116, 248, 753, 249, 187, 549, 106, 170, 86, 71, 987, 46, 555, 441, 678, 501, 230, 351, 149, 268, 532, 365, 581, 181, 125, 405, 660, 517, 8

shuffle_num =  40 search_num =  200 max_iterations =  300
best_value =  99.0 [6, 6, 15, 2, 1, 5, 5, 7, 1, 2, 1, 20, 3, 16, 2, 85, 13, 6, 10, 199, 2, 126, 116, 3, 6, 298, 2, 21, 6, 5, 1, 2, 11, 2, 11, 9, 124, 209, 174, 1, 214, 13, 311, 139, 2, 18, 180, 2, 5, 9, 7, 4, 15, 1, 174, 3, 4, 18, 4, 183, 19, 8, 2, 234, 6, 7, 3, 11, 120, 314, 2, 9, 7, 4, 8, 16, 8, 10, 11, 228, 14, 1, 7, 26, 8, 23, 458, 2, 275, 2, 4, 41, 6, 4, 8, 1, 204, 4, 98, 34, 35, 9, 1, 167, 151, 2, 8, 4, 153, 1, 6, 5, 5, 18, 29, 198, 128, 8, 1, 5, 7, 7, 3, 7, 4, 9, 4, 12, 2, 165, 1, 10, 147, 4, 3, 7, 1, 3, 1, 14, 6, 37, 16, 138, 5, 9, 7, 2, 7, 16, 4, 1, 205, 10, 1, 17, 14, 4, 350, 41, 21, 2, 5, 250, 90, 2, 7, 2, 6, 7, 15, 4, 1, 364, 11, 158, 6, 10, 3, 152, 76, 2, 5, 92, 102, 1, 3, 39, 2, 1, 7, 17, 12, 191, 152, 4, 102, 9, 86, 1, 1, 2, 3, 7, 405, 7, 9, 1, 18, 9, 78, 13, 16, 147, 169, 43, 17, 245, 125, 7, 9, 3, 4, 12, 11, 133, 430, 279, 12, 1, 10, 7, 8, 116, 4, 88, 1, 1, 6, 12, 2, 5, 13, 4, 6, 138, 4, 1, 7, 188, 3, 11, 1, 135

shuffle_num =  40 search_num =  200 max_iterations =  300
best_value =  224.0 [43, 4, 29, 33, 7, 17, 71, 3, 3, 34, 7, 32, 2, 38, 51, 2, 1, 13, 37, 4, 30, 57, 24, 34, 3, 4, 4, 36, 63, 44, 21, 9, 7, 2, 18, 8, 7, 27, 4, 76, 43, 78, 13, 28, 16, 23, 3, 47, 13, 9, 17, 26, 1, 2, 37, 3, 27, 4, 50, 11, 32, 1, 27, 26, 14, 16, 9, 3, 66, 23, 25, 12, 6, 50, 13, 47, 1, 1, 10, 37, 66, 5, 13, 47, 15, 21, 16, 66, 71, 43, 13, 14, 3, 8, 26, 20, 17, 66, 3, 1, 32, 2, 73, 21, 8, 24, 19, 32, 4, 7, 27, 60, 4, 8, 5, 61, 59, 19, 34, 8, 5, 11, 29, 15, 54, 46, 12, 3, 61, 13, 2, 30, 16, 5, 7, 1, 95, 5, 10, 98, 13, 14, 14, 12, 6, 6, 7, 24, 20, 47, 1, 5, 26, 73, 2, 2, 32, 13, 13, 5, 30, 74, 25, 6, 23, 5, 20, 6, 20, 4, 45, 12, 74, 11, 24, 12, 26, 6, 6, 12, 1, 33, 9, 11, 5, 13, 56, 12, 1, 1, 4, 16, 55, 7, 62, 11, 8, 22, 32, 14, 20, 8, 30, 21, 22, 2, 7, 99, 13, 29, 15, 7, 20, 31, 25, 5, 18, 22, 39, 10, 18, 21, 8, 5, 8, 26, 5, 39, 46, 32, 6, 4, 24, 38, 11, 68, 11, 22, 47, 29, 15, 27, 11, 22, 81, 12, 3, 11, 31, 23, 4, 38

In [370]:
import math

class SimulatedAnnealing:
    #Here we get our parameters and our input
    def __init__(self, stock_length, initial_solution, max_iterations, temperature, cooling_rate, search_num):
        self.stock_length = stock_length
        self.current_solution = initial_solution
        self.best_solution = initial_solution
        self.max_iterations = max_iterations
        self.temperature = temperature
        self.cooling_rate = cooling_rate
        self.search_num = search_num
    #Here we search all neighbors of a point for at most search_num times
    #and we return first better point we get and if we don't get a better neighbor
    #we return a random point as our point's neighbor
    def best_neighbor_main(self, current_solution):
        current_solution_fitness = self.fitness(current_solution)
        last_solution = [current_solution[i] for i in range(len(current_solution))]
        
        for _ in range(self.search_num):
            i = random.randint(0, len(last_solution) - 1)
            j = random.randint(0, len(last_solution) - 1)
            last_solution[i], last_solution[j] = last_solution[j], last_solution[i]
            if(self.fitness(last_solution) < current_solution_fitness):
                return last_solution
            last_solution[i], last_solution[j] = last_solution[j], last_solution[i]
        
        random.shuffle(last_solution)
        return last_solution
    #Here we calculate how much of a waste we will have if we cut in our point's order
    def fitness(self, current_solution):
        waste = 0
        left = self.stock_length
        for i in range(len(current_solution)):
            cur = current_solution[i]
            if(cur <= left):
                left -= cur
            else:
                waste += left
                left = self.stock_length - cur
        waste += left
        
        return waste
    #Here we run our simulated annealing algorithm and we keep going to better points
    #and with probablity of e ^ (((current_value - new_value) / temperature)) we go to a bad point
    #and we cool our temperature with each move we do and we keep a record of our best point to return it
    #as our answer
    def run(self):
        cnt_local = 0
        for iteration in range(self.max_iterations):
            new_solution = self.best_neighbor_main(self.current_solution)

            current_value = self.fitness(self.current_solution)
            new_value = self.fitness(new_solution)
            
            if (new_value < current_value):
                self.current_solution = new_solution
            else:
                P = math.exp((current_value - new_value) / self.temperature)
                if(P < 0.001):
                    cnt_local += 1
                    if(cnt_local > 5):
                        break
                if(random.random() <= P):
                    self.current_solution = new_solution
                    
            if self.fitness(self.current_solution) < self.fitness(self.best_solution):
                self.best_solution = self.current_solution
            
            self.temperature *= self.cooling_rate

        return self.best_solution, self.fitness(self.best_solution)


In [394]:
import re
#Here we read all our inputs and based on our prameter sets
#we run our Simulated annealing algorithm on them and find best results
targets = [55, 79, 114, 234]
for i in range(4):
    with open('input' + chr(49 + i) + '.stock', 'r') as file:
        inp = file.read()
        inp = re.split(r'[,\s\n]+', inp)
        
        stock_length = int(inp[2])
        requests = [int(inp[j]) for j in range(inp.index('Requests:') + 1, inp.index('Answer:'))]
        
        max_iterations = 300
        temperature = 100000
        
        A = [(10, 500, 0.98), (20, 400, 0.96), (30, 300, 0.94)]
        for (shuffle_num, search_num, cooling_rate) in A:
            best_value = 1000000000
            best_solution = []
            for i in range(shuffle_num):
                simulated_annealing = SimulatedAnnealing(stock_length, requests, max_iterations, temperature, cooling_rate, search_num)
                final_solution, final_value = simulated_annealing.run()
                final_value = (final_value + sum(requests)) / stock_length
                
                if(final_value < best_value):
                    best_value = final_value
                    best_solution = final_solution
                random.shuffle(requests)
                
            print("shuffle_num = ", shuffle_num, "cooling_rate = ", cooling_rate, "search_num = ", search_num)
            print("best_value = ", best_value, best_solution)

shuffle_num =  10 cooling_rate =  0.98 search_num =  500
best_value =  53.0 [370, 549, 118, 78, 123, 135, 186, 264, 106, 517, 371, 125, 266, 286, 99, 43, 501, 351, 86, 689, 232, 365, 557, 286, 45, 402, 441, 368, 463, 266, 278, 218, 181, 170, 295, 33, 648, 60, 525, 292, 988, 92, 660, 241, 312, 506, 518, 117, 301, 753, 92, 967, 581, 409, 246, 672, 788, 686, 248, 460, 283, 249, 211, 312, 424, 678, 230, 333, 544, 119, 354, 627, 555, 125, 315, 224, 662, 515, 306, 337, 457, 171, 405, 268, 88, 145, 805, 149, 868, 109, 507, 414, 788, 412, 532, 251, 632, 70, 75, 592, 115, 933, 284, 61, 356, 148, 69, 914, 144, 106, 79, 18, 495, 106, 107, 23, 557, 437, 71, 149, 53, 80, 987, 46, 84, 618, 116, 609, 187, 149, 716, 280, 106, 753, 557, 346, 88, 126, 653, 180]
shuffle_num =  20 cooling_rate =  0.96 search_num =  400
best_value =  53.0 [278, 648, 43, 312, 312, 180, 118, 88, 115, 788, 868, 517, 107, 967, 371, 248, 354, 753, 70, 123, 753, 557, 437, 988, 181, 218, 501, 106, 33, 507, 284, 23, 805, 286, 544,

shuffle_num =  20 cooling_rate =  0.96 search_num =  400
best_value =  101.0 [24, 2, 10, 10, 2, 3, 25, 15, 2, 2, 3, 9, 160, 1, 1, 3, 2, 3, 1, 2, 5, 204, 9, 1, 7, 76, 2, 255, 1, 6, 134, 108, 5, 59, 9, 125, 67, 1, 314, 1, 13, 170, 4, 11, 201, 279, 8, 7, 186, 9, 118, 5, 3, 7, 49, 43, 11, 1, 6, 9, 6, 8, 7, 1, 23, 4, 5, 343, 79, 7, 6, 9, 7, 1, 10, 12, 5, 10, 2, 5, 7, 17, 98, 16, 3, 1, 23, 4, 128, 52, 7, 110, 245, 2, 7, 75, 132, 139, 5, 5, 1, 68, 271, 2, 239, 3, 188, 1, 350, 2, 9, 6, 2, 6, 133, 237, 1, 5, 1, 5, 16, 2, 12, 66, 169, 290, 5, 4, 6, 8, 359, 5, 2, 3, 6, 7, 1, 4, 1, 96, 8, 11, 361, 40, 6, 433, 10, 2, 6, 225, 5, 4, 1, 91, 5, 1, 2, 152, 13, 7, 4, 9, 12, 1, 12, 1, 24, 1, 7, 7, 18, 1, 5, 2, 1, 339, 5, 12, 5, 6, 3, 15, 1, 9, 2, 5, 16, 19, 6, 250, 43, 3, 135, 21, 11, 2, 2, 24, 6, 263, 5, 4, 3, 6, 9, 124, 399, 2, 7, 22, 11, 3, 5, 49, 10, 8, 4, 90, 134, 1, 6, 7, 227, 288, 2, 12, 198, 6, 1, 10, 1, 4, 8, 20, 9, 1, 9, 18, 4, 34, 281, 151, 3, 2, 17, 280, 7, 12, 12, 2, 5, 11, 14, 255, 6, 158, 1

shuffle_num =  20 cooling_rate =  0.96 search_num =  400
best_value =  227.0 [6, 8, 10, 3, 5, 6, 27, 9, 11, 63, 12, 15, 8, 9, 13, 35, 18, 51, 6, 5, 6, 20, 4, 16, 2, 12, 6, 9, 4, 32, 18, 9, 73, 8, 2, 24, 44, 93, 8, 51, 18, 1, 62, 4, 26, 29, 6, 43, 7, 61, 8, 2, 15, 67, 32, 23, 35, 37, 5, 10, 31, 18, 20, 3, 9, 71, 59, 22, 80, 23, 66, 18, 29, 12, 2, 24, 8, 4, 43, 29, 19, 1, 7, 16, 33, 36, 51, 21, 12, 2, 27, 20, 51, 78, 2, 5, 23, 14, 2, 1, 15, 27, 16, 38, 13, 20, 15, 11, 43, 19, 15, 15, 20, 45, 1, 19, 26, 38, 17, 9, 10, 20, 26, 21, 20, 25, 21, 25, 14, 2, 22, 67, 15, 2, 5, 68, 10, 10, 28, 56, 16, 52, 59, 40, 28, 5, 58, 1, 2, 13, 62, 19, 12, 16, 20, 3, 3, 54, 37, 89, 9, 7, 2, 15, 10, 3, 27, 5, 25, 2, 65, 5, 11, 14, 37, 26, 21, 68, 16, 18, 10, 29, 8, 30, 14, 33, 44, 3, 72, 18, 4, 5, 50, 20, 2, 78, 18, 90, 7, 5, 6, 25, 61, 2, 72, 1, 45, 13, 2, 8, 12, 1, 3, 26, 32, 4, 2, 4, 24, 26, 52, 34, 6, 51, 1, 56, 70, 21, 36, 52, 7, 12, 56, 7, 19, 43, 35, 6, 6, 4, 1, 89, 2, 4, 7, 66, 29, 37, 3, 9, 20, 46, 